In [1]:
import requests
import pandas as pd
import json
import datetime
import numpy as np

# for geocoding API
import http.client, urllib.parse

In [2]:
# Laurina LaStella:
# When 'region' is included, there are errors I can't explain or solve.
# But the 'query' variable allows city and state name combined.

In [3]:
country_input = (input("Enter a Country using a 2-letter country code (Example: US): "))
# region_input = (input("Enter a Region (City, County, State, etc.): "))
query_input = (input("Enter a Location (City, Zip Code, etc.): "))

print(
    country_input,
#     region_input,
    query_input
)

Enter a Country using a 2-letter country code (Example: US): fr
Enter a Location (City, Zip Code, etc.): paris
fr paris


In [4]:
conn = http.client.HTTPConnection('api.positionstack.com')
params = urllib.parse.urlencode({
    'access_key': '1fbf7de4ce05274e9b9005659970c429',
   
    # Variables get live user input above.
    'country': country_input,
    
#     Errors, unknown reason.
#     'region': region_input,
    
#     Can include city and state.
    'query': query_input,
        
    # "limit" of 1 ensures only 1 Lat/Lon will be passed below.    
    'limit': 1
    })

conn.request('GET', '/v1/forward?{}'.format(params))
res = conn.getresponse()
geo = res.read()
geo_str = (geo.decode('utf-8'))

###########################################################
# Still needs error handling.

# Convert string to json (which is a dictionary).
geo_json = json.loads(geo_str)

# extract latitude
geo_json_lat = geo_json['data']
var_lat = []
for x in geo_json_lat:
    var_lat.append((x['latitude']))
    var_lat_solo = str(var_lat[0])

# extract longitude
geo_json_lon = geo_json['data']
var_lon = []
for x in geo_json_lon:
    var_lon.append((x['longitude']))
    var_lon_solo = str(var_lon[0])

# extract geoc label
geo_json_label = geo_json['data']
geo_label = []
for x in geo_json_label:
    geo_label.append((x['label']))
    geo_label_solo = str(geo_label[0])
    
print(geo.decode('utf-8'))

{"data":[{"latitude":48.858705,"longitude":2.342865,"type":"locality","name":"Paris","number":null,"postal_code":null,"street":null,"confidence":1,"region":"Paris","region_code":"VP","county":null,"locality":"Paris","administrative_area":"Paris","neighbourhood":null,"country":"France","country_code":"FRA","continent":"Europe","label":"Paris, France"}]}


In [5]:
type(geo_label_solo)

str

In [6]:
geo_json

{'data': [{'latitude': 48.858705,
   'longitude': 2.342865,
   'type': 'locality',
   'name': 'Paris',
   'number': None,
   'postal_code': None,
   'street': None,
   'confidence': 1,
   'region': 'Paris',
   'region_code': 'VP',
   'county': None,
   'locality': 'Paris',
   'administrative_area': 'Paris',
   'neighbourhood': None,
   'country': 'France',
   'country_code': 'FRA',
   'continent': 'Europe',
   'label': 'Paris, France'}]}

In [7]:
geo_str

'{"data":[{"latitude":48.858705,"longitude":2.342865,"type":"locality","name":"Paris","number":null,"postal_code":null,"street":null,"confidence":1,"region":"Paris","region_code":"VP","county":null,"locality":"Paris","administrative_area":"Paris","neighbourhood":null,"country":"France","country_code":"FRA","continent":"Europe","label":"Paris, France"}]}'

In [8]:
type(var_lat_solo)

str

In [9]:
type(var_lon_solo)

str

In [10]:
var_lat_solo

'37.865832'

In [11]:
var_lon_solo

'-122.297864'

In [8]:
# this function builds the url for the API call to retrieve HOURLY weather data 
# from https://archive-api.open-meteo.com/v1/era5
# for given lattitue/longitude coordinates, for the given years;

def get_hourly_weather_url(var_lat_solo, var_lon_solo, start_year, end_year):
    url_snip_hr = {}
    # Break URL into snippets to be assembled below, hourly:
    url_snip_hr[1] = "https://archive-api.open-meteo.com/v1/era5?latitude=" 
    url_snip_hr[2] = var_lat_solo
    url_snip_hr[3] = "&longitude="
    url_snip_hr[4] = var_lon_solo
    url_snip_hr[5] = "&start_date="
    url_snip_hr[6] = str(start_year)
    url_snip_hr[7] = "-01-01&end_date="  # month and day of start_year
    url_snip_hr[8] = str(end_year)
    url_snip_hr[9] = "-12-31&hourly="   # elements, hourly
    url_snip_hr[10] = "temperature_2m,"   # data element
    url_snip_hr[11] = "relativehumidity_2m,"   # data element
    url_snip_hr[12] = "rain,"   # data element
    url_snip_hr[13] = "snowfall,"   # data element
    url_snip_hr[14] = "cloudcover,"   # data element
    url_snip_hr[15] = "windspeed_10m,"   # data element
    url_snip_hr[16] = "winddirection_10m"   # data element
    url_snip_hr[17] = "&timezone=America%2FLos_Angeles"   # time zone
    url_snip_hr[18] = "&temperature_unit=fahrenheit"   # temp unit
    url_snip_hr[19] = "&windspeed_unit=mph"   # windspeed unit
    url_snip_hr[20] = "&precipitation_unit=inch"   # precip unit
    weather_url_hr = ""
    for i in range(20):
        weather_url_hr += url_snip_hr[i+1]
    return weather_url_hr   

In [9]:
# this function builds the url for the API call to retrieve DAILY weather data 
# from https://archive-api.open-meteo.com/v1/era5
# for given lattitue/longitude coordinates, for the given years;

def get_daily_weather_url(var_lat_solo, var_lon_solo, start_year, end_year):
    url_snip_dy = {}
    # Break URL into snippets to be assembled below, daily:
    url_snip_dy[1] = "https://archive-api.open-meteo.com/v1/era5?latitude=" 
    url_snip_dy[2] = var_lat_solo   # latitude, input from previous json
    url_snip_dy[3] = "&longitude="
    url_snip_dy[4] = var_lon_solo   # longitude, input from previous json
    url_snip_dy[5] = "&start_date="
    url_snip_dy[6] = str(start_year)
    url_snip_dy[7] = "-01-01&end_date="  # month and day of start_year
    url_snip_dy[8] = str(end_year)
    url_snip_dy[9] = "-12-31&daily="   # elements, daily
    url_snip_dy[10] = "temperature_2m_max,"   # data element
    url_snip_dy[11] = "temperature_2m_min,"   # data element
    url_snip_dy[12] = "rain_sum,"   # data element
    url_snip_dy[13] = "snowfall_sum,"   # data element
    url_snip_dy[14] = "precipitation_hours"   # data element
    url_snip_dy[15] = "&timezone=America%2FLos_Angeles"   # time zone
    url_snip_dy[16] = "&temperature_unit=fahrenheit"   # temp unit
    url_snip_dy[17] = "&windspeed_unit=mph"   # windspeed unit
    url_snip_dy[18] = "&precipitation_unit=inch"   # precip unit
    weather_url_dy = ""
    for i in range(18):
        weather_url_dy += url_snip_dy[i+1]
    return weather_url_dy

In [10]:
# daily_or_hourly should be a string
#  ### needs better error-handling ###

def get_weather_url(var_lat_solo, var_lon_solo, start_year, end_year, daily_or_hourly):
    if daily_or_hourly == 'daily':
        return get_daily_weather_url(var_lat_solo, var_lon_solo, start_year, end_year)
    elif daily_or_hourly== 'hourly':
        return get_hourly_weather_url(var_lat_solo, var_lon_solo, start_year, end_year)
    else:
        return 'error'

In [11]:
# # testing: 'print' makes them clickable!! so go look at the data :)

# print( get_weather_url(35.695126, 139.75391, 2010, 2019,'daily') )
# print( get_weather_url(35.695126, 139.75391, 2010, 2019,'hourly') )


print( get_weather_url(var_lat_solo, var_lon_solo, 2010, 2019,'daily') )
print( get_weather_url(var_lat_solo, var_lon_solo, 2010, 2019,'hourly') )

https://archive-api.open-meteo.com/v1/era5?latitude=48.858705&longitude=2.342865&start_date=1981-01-01&end_date=2018-12-31&daily=temperature_2m_max,temperature_2m_min,rain_sum,snowfall_sum,precipitation_hours&timezone=America%2FLos_Angeles&temperature_unit=fahrenheit&windspeed_unit=mph&precipitation_unit=inch
https://archive-api.open-meteo.com/v1/era5?latitude=48.858705&longitude=2.342865&start_date=2010-01-01&end_date=2019-12-31&hourly=temperature_2m,relativehumidity_2m,rain,snowfall,cloudcover,windspeed_10m,winddirection_10m&timezone=America%2FLos_Angeles&temperature_unit=fahrenheit&windspeed_unit=mph&precipitation_unit=inch


In [16]:
# this function retrieves weather data for given lattitue/longitude coordinates
# from https://archive-api.open-meteo.com/v1/era5
# for the given years; 
# e.g. if start_year=2010 and end_year=2020, 11 years of data are retrieved,
# starting 2010-01-01 and ending 2020-12-31, inclusive
#   ### valid years ###
# again, daily_or_hourly must be one of the two strings

def get_weather(latitude, longitude, start_year, end_year, daily_or_hourly):
    url = get_weather_url(var_lat_solo, var_lon_solo, start_year, end_year, daily_or_hourly)
    # Data comes in as one long string:
    weather_hr_str = requests.get(url).text
    # So convert string to dictionary.
    weather_hr_json = json.loads(weather_hr_str)
    # now turn dictionary into dataframe
    weather_raw = pd.DataFrame.from_records(weather_hr_json[daily_or_hourly])
    # convert the provided ISO string 'time' into a 'pure_date' in python datetime format
    # for aggregating and joining with daily data
    weather_raw["pure_date"] = weather_raw['time'].map(lambda x: 
                                                        datetime.datetime.fromisoformat(x[0:10]))
    # forget the string with the hour
    weather_raw.drop('time', axis=1, inplace=True)
    return weather_raw

In [17]:
# testing this should complete within a second or five, without errors
#  ### testing best practices: how do i display expected output?? 
#  ### which is a better test: test_df.info() or test_df.describe()?

daily_test_df = get_weather(var_lat_solo, var_lon_solo, 2010, 2019, 'daily')
daily_test_df.info()

hourly_test_df = get_weather(var_lat_solo, var_lon_solo, 2010, 2019, 'hourly')
hourly_test_df.info()

# note that wind_direction in hourly is null when windspeed is 0
# we do nothing with wind direction, so don't bother fixing it

# gaierror means check your internet connection

# these dataframes are used in later testing!

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3652 entries, 0 to 3651
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   precipitation_hours  3652 non-null   float64       
 1   rain_sum             3652 non-null   float64       
 2   snowfall_sum         3652 non-null   float64       
 3   temperature_2m_max   3652 non-null   float64       
 4   temperature_2m_min   3652 non-null   float64       
 5   pure_date            3652 non-null   datetime64[ns]
dtypes: datetime64[ns](1), float64(5)
memory usage: 171.3 KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87648 entries, 0 to 87647
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   cloudcover           87648 non-null  int64         
 1   rain                 87648 non-null  float64       
 2   relativehumidity_2m  87648 non-null 

In [18]:
var_lat_solo

'37.865832'

In [19]:
var_lon_solo

'-122.297864'

In [20]:
geo_label

['Berkeley, CA, USA']

In [21]:
# this auxillary function will be used to aggreagte hourly data into daily
# for example, getting the temperature of 6th hottest hour answers questions like:
# Were there at least 6 hours above 80F? and Were there at least 18 hours below 32F?

def enth(x, n):
    return x.sort_values().iloc[n]

In [22]:
# this function takes in the hourly and daily weather dataframes made by get_weather
#  ### behaviour if different time periods ###
# the returned dataframe is indexed by pure_date, so same number of rows as daily input
# the returned dataframe has all the columns of the daily input,
# plus a bunch of aggregates of data from hourly
# yes, max_wind = wind_high and temp_high is also a rename; someday, we fix this inefficiency

def agg_hourly_and_daily(hourly_df, daily_df):
    ## BEWARE arguments are passed by reference, don't mess with them ! ##
    output_df = hourly_df.groupby('pure_date').agg(
    # historical statistics (adjectives): humid_avg, wind_high, cloud_avg, temp_6
        humid_avg=('relativehumidity_2m', np.mean),
        wind_high = ('windspeed_10m', np.max),
        cloud_avg=('cloudcover', np.mean),
        temp_6= ('temperature_2m', lambda x: enth(x,18)),
    # machine learning: avg_humidity, median_wind, max_wind, cloud_4, cloud_12, cloud_20
        avg_humidity=('relativehumidity_2m', np.mean),
        median_wind = ('windspeed_10m', np.median),
        max_wind = ('windspeed_10m', np.max),
        cloud_4 = ('cloudcover', lambda x: enth(x,3)),
        cloud_12 = ('cloudcover', lambda x: enth(x,11)),
        cloud_20 = ('cloudcover', lambda x: enth(x,19)) )
    output_df = output_df.join(daily_df.set_index('pure_date'))
    output_df['temp_high'] = output_df['temperature_2m_max']
    # i want 'pure_date' to go back to being a regular column
    output_df.reset_index(inplace=True)
    return output_df

In [23]:
# testing, using the hourly_test_df from previous test
#  ### again, how to display expected output? ###

clean_df1 = agg_hourly_and_daily(hourly_test_df, daily_test_df)
# Adding 'geo_tag' from geocoding API above.
clean_df = clean_df1.assign(geo_tag = geo_label_solo)
clean_df.info()
clean_df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3652 entries, 0 to 3651
Data columns (total 18 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   pure_date            3652 non-null   datetime64[ns]
 1   humid_avg            3652 non-null   float64       
 2   wind_high            3652 non-null   float64       
 3   cloud_avg            3652 non-null   float64       
 4   temp_6               3652 non-null   float64       
 5   avg_humidity         3652 non-null   float64       
 6   median_wind          3652 non-null   float64       
 7   max_wind             3652 non-null   float64       
 8   cloud_4              3652 non-null   int64         
 9   cloud_12             3652 non-null   int64         
 10  cloud_20             3652 non-null   int64         
 11  precipitation_hours  3652 non-null   float64       
 12  rain_sum             3652 non-null   float64       
 13  snowfall_sum         3652 non-nul

,humid_avg,wind_high,cloud_avg,temp_6,avg_humidity,median_wind,max_wind,cloud_4,cloud_12,cloud_20,precipitation_hours,rain_sum,snowfall_sum,temperature_2m_max,temperature_2m_min,temp_high
count,3652.000000,3652.000000,3652.000000,3652.000000,3652.000000,3652.000000,3652.000000,3652.000000,3652.000000,3652.000000,3652.000000,3652.000000,3652.0,3652.000000,3652.000000,3652.000000
mean,72.392479,9.791128,36.580219,64.812760,72.392479,5.869578,9.791128,13.841731,30.983844,60.900055,1.729463,0.051443,0.0,68.379573,51.330120,68.379573
std,11.234203,3.344862,26.157578,8.088679,11.234203,2.497518,3.344862,22.437121,31.783557,34.256650,4.422766,0.179160,0.0,8.791370,6.314325,8.791370
min,18.041667,2.900000,0.000000,42.700000,18.041667,1.600000,2.900000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,46.200000,29.400000,46.200000
25%,67.489583,7.500000,15.250000,58.800000,67.489583,3.800000,7.500000,0.000000,3.000000,29.000000,0.000000,0.000000,0.0,61.800000,47.200000,61.800000
50%,74.041667,9.500000,33.041667,64.700000,74.041667,5.450000,9.500000,2.000000,22.000000,72.000000,0.000000,0.000000,0.0,68.100000,52.500000,68.100000
75%,79.125000,11.700000,53.385417,70.300000,79.125000,7.500000,11.700000,20.000000,50.000000,90.000000,0.000000,0.000000,0.0,74.100000,55.700000,74.100000
max,95.916667,23.400000,100.000000,100.100000,95.916667,17.700000,23.400000,100.000000,100.000000,100.000000,24.000000,2.580000,0.0,104.900000,71.100000,104.900000


In [24]:
clean_df

,pure_date,humid_avg,wind_high,cloud_avg,temp_6,avg_humidity,median_wind,max_wind,cloud_4,cloud_12,cloud_20,precipitation_hours,rain_sum,snowfall_sum,temperature_2m_max,temperature_2m_min,temp_high,geo_tag
0,2010-01-01,87.458333,5.1,96.750000,57.0,87.458333,3.25,5.1,90,100,100,4.0,0.08,0.0,58.1,51.3,58.1,"Berkeley, CA, USA"
1,2010-01-02,91.916667,4.7,93.500000,56.1,91.916667,2.90,4.7,86,100,100,0.0,0.00,0.0,58.0,48.2,58.0,"Berkeley, CA, USA"
2,2010-01-03,92.125000,8.6,58.416667,50.8,92.125000,4.50,8.6,24,42,100,0.0,0.00,0.0,53.7,44.1,53.7,"Berkeley, CA, USA"
3,2010-01-04,90.041667,6.3,32.708333,50.8,90.041667,4.65,6.3,1,22,70,0.0,0.00,0.0,54.4,43.9,54.4,"Berkeley, CA, USA"
4,2010-01-05,86.875000,4.8,47.083333,51.2,86.875000,3.50,4.8,28,30,100,0.0,0.00,0.0,54.2,42.0,54.2,"Berkeley, CA, USA"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3647,2019-12-27,72.375000,6.7,8.166667,53.8,72.375000,3.95,6.7,0,4,18,0.0,0.00,0.0,59.1,38.5,59.1,"Berkeley, CA, USA"
3648,2019-12-28,83.583333,5.2,40.708333,52.7,83.583333,2.90,5.2,0,33,79,0.0,0.00,0.0,57.9,37.6,57.9,"Berkeley, CA, USA"
3649,2019-12-29,90.833333,8.5,100.000000,54.3,90.833333,5.10,8.5,100,100,100,10.0,0.24,0.0,56.6,46.1,56.6,"Berkeley, CA, USA"
3650,2019-12-30,82.916667,8.9,19.958333,54.1,82.916667,3.60,8.9,0,0,40,1.0,0.00,0.0,58.8,44.5,58.8,"Berkeley, CA, USA"


## Getting adjectives for historical summary

Now we 
 - filter by user-requested dates;
 - get boolean columns for adjectives, and 
 - compute rates/likelihoods for each adjective.

The next function takes in a date interval, generates the same intervals in previous years, and returns the starts and ends of those intervals, starting with the original.

In [153]:
# start_date, end_date should be datetime; 
# years >= 0 should integer; if years=0, ([start_date*], [end_date*]) is returned
# (*) with feb-29 removed


def backdate(start_date, end_date, years):
    # replace start_date feb-29 by feb-28
    if (start_date.month == 2 & start_date.day == 29):
        start_date = start_date - datetime.timedelta(days = 1)
    # replace end_date feb-29 by mar-01
    if (end_date.month == 2 & end_date.day == 29):
        end_date = end_date + datetime.timedelta(days = 1)
    starts = [start_date]
    ends = [end_date]
    for i in range(years):
        start_date = start_date.replace(year = start_date.year -1)
        end_date = end_date.replace(year = end_date.year -1)
        starts.append(start_date)
        ends.append(end_date)
    return (starts, ends)
    

In [154]:
# testing: expected output
# ([datetime.datetime(2023, 2, 25, 0, 0),
#   datetime.datetime(2022, 2, 25, 0, 0),
#   datetime.datetime(2021, 2, 25, 0, 0),
#   datetime.datetime(2020, 2, 25, 0, 0)],
#  [datetime.datetime(2023, 3, 5, 0, 0),
#   datetime.datetime(2022, 3, 5, 0, 0),
#   datetime.datetime(2021, 3, 5, 0, 0),
#   datetime.datetime(2020, 3, 5, 0, 0)])

# first two are used in further testing
star = datetime.datetime.fromisoformat('2023-02-25')
endd = datetime.datetime.fromisoformat('2023-03-05')

starts_ends = backdate(star, endd, 3)
starts_ends

([datetime.datetime(2023, 2, 25, 0, 0),
  datetime.datetime(2022, 2, 25, 0, 0),
  datetime.datetime(2021, 2, 25, 0, 0),
  datetime.datetime(2020, 2, 25, 0, 0)],
 [datetime.datetime(2023, 3, 5, 0, 0),
  datetime.datetime(2022, 3, 5, 0, 0),
  datetime.datetime(2021, 3, 5, 0, 0),
  datetime.datetime(2020, 3, 5, 0, 0)])

The next function ingests a dataframe with a column 'pure_date', along with a date interval, and the number of years to go back by; and returns the rows of the input dataframe where the pure_date falls within one of the intervals.

In [155]:
def date_period_filter(daf, start_date, end_date, years):
    starts_ends = backdate(start_date, end_date, years)
    # for each backdated interval, test whether pure_date falls in it, then
    # add up the booleans, get 0 if all false, more otherwise
    mask = sum( (daf['pure_date'] >= starts_ends[0][i]) &
               (daf['pure_date'] <= starts_ends[1][i]) for i in range(years) )
    # if interval is too big, backdated intervals may overlap, but we don't want 2 as a value
    # also, returns it to boolean from integer
    mask = (mask > 0)
    return daf[mask].copy()
    

In [156]:
# testing, used below 
little_df = date_period_filter(clean_df, star, endd, 6)
little_df

,pure_date,humid_avg,wind_high,cloud_avg,temp_6,avg_humidity,median_wind,max_wind,cloud_4,cloud_12,cloud_20,precipitation_hours,rain_sum,snowfall_sum,temperature_2m_max,temperature_2m_min,temp_high,geo_tag
2977,2018-02-25,63.208333,24.6,42.791667,40.5,63.208333,18.05,24.6,0,28,88,0.0,0.00,0.00,43.3,32.9,43.3,"Chicago, IL, USA"
2978,2018-02-26,62.500000,12.7,1.250000,47.9,62.500000,8.65,12.7,0,0,0,0.0,0.00,0.00,51.4,29.3,51.4,"Chicago, IL, USA"
2979,2018-02-27,54.250000,15.7,11.291667,54.9,54.250000,13.65,15.7,0,6,24,0.0,0.00,0.00,58.8,38.8,58.8,"Chicago, IL, USA"
2980,2018-02-28,79.708333,15.4,73.791667,49.0,79.708333,9.40,15.4,36,91,100,3.0,0.06,0.00,55.3,37.2,55.3,"Chicago, IL, USA"
2981,2018-03-01,83.583333,18.1,50.541667,37.8,83.583333,11.35,18.1,1,31,100,11.0,0.77,0.07,39.3,33.0,39.3,"Chicago, IL, USA"
2982,2018-03-02,70.708333,10.0,10.583333,39.3,70.708333,6.30,10.0,0,3,17,0.0,0.00,0.00,41.1,30.6,41.1,"Chicago, IL, USA"
2983,2018-03-03,68.750000,9.4,0.583333,37.6,68.750000,7.95,9.4,0,0,1,0.0,0.00,0.00,39.3,31.8,39.3,"Chicago, IL, USA"
2984,2018-03-04,57.916667,15.9,5.458333,40.0,57.916667,10.40,15.9,0,1,8,0.0,0.00,0.00,42.2,31.9,42.2,"Chicago, IL, USA"
2985,2018-03-05,66.958333,18.3,68.875000,36.6,66.958333,15.10,18.3,34,69,100,9.0,0.09,3.01,37.7,32.4,37.7,"Chicago, IL, USA"
3342,2019-02-25,50.583333,20.4,61.791667,21.4,50.583333,7.60,20.4,24,72,89,0.0,0.00,0.14,21.8,8.3,21.8,"Chicago, IL, USA"


## Step 2: getting adjectives for historical summary statistics.

In [157]:
# ideally, the definitions of adjectives should be stored separately and passed to this function,
# for easier future adjustment; but good enough for now

def add_bool_col_for_adj(daf):
    daf_with_bool = daf.copy()
    
    # temp
    # freezing = >19hours with <32F: Use "temp_6" (6th highest temp of day) as cutoff.
    # cold = high <60F
    # hot = >6hours with >80F
    # warm = leftovers
    daf_with_bool['freezing'] = (daf_with_bool['temp_6'] <= 32)
    daf_with_bool['cold'] = (daf_with_bool['temp_high'] <= 60) & (daf_with_bool['freezing'] == 0)
    daf_with_bool['hot'] = (daf_with_bool['temp_6'] >= 80)
    daf_with_bool['warm'] = (daf_with_bool['freezing'] == 0) & (daf_with_bool['cold'] == 0) & (daf_with_bool['hot'] == 0)

    # clouds
    # average hourly "percent cloud cover"
    # clear = <=33%
    # cloudy = >= 67%
    # partly_cloudy = leftovers
    daf_with_bool['clear'] = (daf_with_bool['cloud_avg'] <= 30)
    daf_with_bool['cloudy'] = (daf_with_bool['cloud_avg'] >= 60)
    daf_with_bool['partly_cloudy'] = (daf_with_bool['clear'] == 0) & (daf_with_bool['cloudy'] == 0)

    # rain
    # not_rainy = 0 hours, or  <2.5 mm total rain (Laurina changing to 0.5 inches)
    # very_rainy = >=6 hour and >=10mm (Laurina changing to 2 inches)
    # lightly_rainy = leftovers
    daf_with_bool['not_rainy'] = (daf_with_bool['rain_sum'] <= 0.5) | (daf_with_bool['precipitation_hours'] == 0)
    daf_with_bool['very_rainy'] = (daf_with_bool['rain_sum'] >= 2) | (daf_with_bool['precipitation_hours'] >= 6)
    daf_with_bool['lightly_rainy'] = (daf_with_bool['not_rainy'] == 0) & (daf_with_bool['very_rainy'] == 0)

    # snow
    # data used: total snow for the day
    # not_snowy = 0
    # very_snowy = >6"
    # lightly_snowy = >0
    # (unlike rain, we ignore hours)   
    daf_with_bool['not_snowy'] = (daf_with_bool['snowfall_sum'] == 0)
    daf_with_bool['very_snowy'] = (daf_with_bool['snowfall_sum'] >= 6)
    daf_with_bool['lightly_snowy'] = (daf_with_bool['not_snowy'] == 0) & (daf_with_bool['very_snowy'] == 0)

    # wind
    # windy = maximum	> ?? mph
    # maybe 8? mayb 12  
    daf_with_bool['windy'] = (daf_with_bool['wind_high'] >= 12)

    # humidity
    # average hourlies, then:
    # low_humidity = <30%
    # high_humidity = >60%
    # medium_humidity = leftovers
    daf_with_bool['low_humidity'] = (daf_with_bool['humid_avg'] <= 30)
    daf_with_bool['high_humidity'] = (daf_with_bool['humid_avg'] >= 60)
    daf_with_bool['medium_humidity'] = (daf_with_bool['low_humidity'] == 0) & (daf_with_bool['high_humidity'] == 0)

    return daf_with_bool
    

In [158]:
#testing

little_with_bool = add_bool_col_for_adj(little_df)
little_with_bool

# note that a day-long drizzle is both very_rainy and not_rainy

,pure_date,humid_avg,wind_high,cloud_avg,temp_6,avg_humidity,median_wind,max_wind,cloud_4,cloud_12,...,not_rainy,very_rainy,lightly_rainy,not_snowy,very_snowy,lightly_snowy,windy,low_humidity,high_humidity,medium_humidity
2977,2018-02-25,63.208333,24.6,42.791667,40.5,63.208333,18.05,24.6,0,28,...,True,False,False,True,False,False,True,False,True,False
2978,2018-02-26,62.500000,12.7,1.250000,47.9,62.500000,8.65,12.7,0,0,...,True,False,False,True,False,False,True,False,True,False
2979,2018-02-27,54.250000,15.7,11.291667,54.9,54.250000,13.65,15.7,0,6,...,True,False,False,True,False,False,True,False,False,True
2980,2018-02-28,79.708333,15.4,73.791667,49.0,79.708333,9.40,15.4,36,91,...,True,False,False,True,False,False,True,False,True,False
2981,2018-03-01,83.583333,18.1,50.541667,37.8,83.583333,11.35,18.1,1,31,...,False,True,False,False,False,True,True,False,True,False
2982,2018-03-02,70.708333,10.0,10.583333,39.3,70.708333,6.30,10.0,0,3,...,True,False,False,True,False,False,False,False,True,False
2983,2018-03-03,68.750000,9.4,0.583333,37.6,68.750000,7.95,9.4,0,0,...,True,False,False,True,False,False,False,False,True,False
2984,2018-03-04,57.916667,15.9,5.458333,40.0,57.916667,10.40,15.9,0,1,...,True,False,False,True,False,False,True,False,False,True
2985,2018-03-05,66.958333,18.3,68.875000,36.6,66.958333,15.10,18.3,34,69,...,True,True,False,False,False,True,True,False,True,False
3342,2019-02-25,50.583333,20.4,61.791667,21.4,50.583333,7.60,20.4,24,72,...,True,False,False,False,False,True,True,False,False,True


Aggregate boolean columns into statistics.

In [159]:
# location is likely to consist of the coutry, city, state info that goes 

def bool_to_stats(daf):
    # list of column names to be aggregated
    bool_cols = ['freezing', 'cold', 'warm', 'hot',
                'not_rainy', 'lightly_rainy', 'very_rainy',
                'not_snowy', 'lightly_snowy', 'very_snowy',
                'windy',
                'low_humidity', 'medium_humidity', 'high_humidity',
                'clear', 'partly_cloudy', 'cloudy']
    # denominators for rate/likelyhood
    row_count = len(daf)
    # initialize storage
    stats_dict = {}
    # aggregate
    for stat in bool_cols:
        stats_dict[stat] = daf[stat].sum() / row_count
    return stats_dict



In [160]:
# testing

bool_to_stats(little_with_bool)

{'freezing': 0.5,
 'cold': 0.5,
 'warm': 0.0,
 'hot': 0.0,
 'not_rainy': 0.9444444444444444,
 'lightly_rainy': 0.0,
 'very_rainy': 0.2222222222222222,
 'not_snowy': 0.5,
 'lightly_snowy': 0.5,
 'very_snowy': 0.0,
 'windy': 0.7777777777777778,
 'low_humidity': 0.0,
 'medium_humidity': 0.3333333333333333,
 'high_humidity': 0.6666666666666666,
 'clear': 0.2777777777777778,
 'partly_cloudy': 0.16666666666666666,
 'cloudy': 0.5555555555555556}

In [ ]:
bob = [1,3]

In [ ]:
sum([i==2 for i in bob])

In [ ]:
b = pd.DataFrame({'id': [1, 2, 9, 8],
                  'is': [1, 3, 5, 8],
                  'val1': ['p', 'q', 'r', 's']})

In [ ]:
b

In [ ]:
b.set_index('id', inplace=True)

In [ ]:
b['id'] = ( b['id'] > 0)

In [ ]:
b

In [ ]:
b['id'] = b['id'].map(lambda x: max(x, 1)

In [ ]:
b.reset_index(inplace=True)

In [ ]:
b

In [ ]:
start_dat = [1,2,5]
end_dat = [3,5,8]

In [ ]:
i=1
joe = (b['id'] >= start_dat[i]) & (b['id'] <= end_dat[i])

In [ ]:
joe

In [ ]:
joe+joe

In [ ]:
sum([joe, joe])

In [ ]:
cat = sum([(b['id'] >= start_dat[i]) & (b['id'] <= end_dat[i]) for i in range(2)])

In [ ]:
cat

In [ ]:
for i in range(0):
    print("cat")